<a href="https://colab.research.google.com/github/Lulens/tp-pod/blob/master/04_Validaci%C3%B3n_de_datos_Individual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   **Año:** 2024
*   **Alumno/a:** María Lucía Báez
*   **Legajo:** 42306816

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [2]:
!pip install pydantic

from pydantic import BaseModel

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [3]:
class Client(BaseModel):
  dni: int
  nationality: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [9]:
class Client(BaseModel):
  dni: int
  nationality: str

user = Client(dni=39755010, nationality='Argentina')
user

Client(dni=39755010, nationality='Argentina')

Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [23]:
from pydantic import field_validator, ValidationError

class Client(BaseModel):
    dni: int
    nationality: str

    @field_validator('dni')
    def dni_must_be_integer(cls, value):
        if isinstance(value, str):
            raise ValueError('dni must be an integer, not a string')
        return value

client = Client(dni="123456789", nationality="Argentine")
client

Client(dni=123456789, nationality='Argentine')

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [25]:
from datetime import datetime

valid_nationalities = ["Argentina", "Brazil", "Chile", "Uruguay"]

class Client(BaseModel):
    dni: int
    nationality: str
    registration_date: datetime

    @field_validator('dni')
    def dni_must_be_positive(cls, value):
        if value < 0:
            raise ValueError('dni must be a positive integer')
        return value

    @field_validator('nationality')
    def nationality_must_be_valid(cls, value):
        if value not in valid_nationalities:
            raise ValueError(f'{value} is not a valid nationality')
        return value

    @field_validator('registration_date')
    def registration_date_must_not_be_in_future(cls, value):
        if value > datetime.now():
            raise ValueError('registration date cannot be in the future')
        return value

client_1 = Client(dni=123456789, nationality="Argentina", registration_date=datetime(2023, 11, 15))
print(client_1)

dni=123456789 nationality='Argentina' registration_date=datetime.datetime(2023, 11, 15, 0, 0)


In [30]:
# Caso 2: DNI negativo (debería fallar)
client_2 = Client(dni=-123456789, nationality="Argentina", registration_date=datetime(2023, 11, 15))
client_2

ValidationError: 1 validation error for Client
dni
  Value error, dni must be a positive integer [type=value_error, input_value=-123456789, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [31]:
# Caso 3: Nacionalidad no válida (debería fallar)
client_3 = Client(dni=123456789, nationality="España", registration_date=datetime(2023, 11, 15))
client_3

ValidationError: 1 validation error for Client
nationality
  Value error, España is not a valid nationality [type=value_error, input_value='España', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [33]:
# Caso 4: Fecha de registro en el futuro (debería fallar)
client_4 = Client(dni=123456789, nationality="Argentina", registration_date=datetime(2025, 11, 15))
client_4

ValidationError: 1 validation error for Client
registration_date
  Value error, registration date cannot be in the future [type=value_error, input_value=datetime.datetime(2025, 11, 15, 0, 0), input_type=datetime]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [39]:
!pip install pandera

import pandas as pd
import pandera as pa
from datetime import datetime

valid_nationalities = ["Argentina", "Brazil", "Chile", "Uruguay"]

class ClientSchema(pa.DataFrameModel):
    dni: pa.typing.Series[int]
    nationality: pa.typing.Series[str]
    registration_date: pa.typing.Series[datetime]

    @pa.check("dni")
    def check_dni_positive(cls, series: pd.Series) -> pd.Series:
        return series > 0

    @pa.check("nationality")
    def check_nationality_valid(cls, series: pd.Series) -> pd.Series:
        return series.isin(valid_nationalities)

    @pa.check("registration_date")
    def check_registration_date(cls, series: pd.Series) -> pd.Series:
        return series <= datetime.now()

data = {
    "dni": [123456789, -123456789, 987654321],
    "nationality": ["Argentina", "España", "Brazil"],
    "registration_date": [
        datetime(2023, 11, 15),
        datetime(2023, 11, 15),
        datetime(2025, 11, 15),
    ],
}

df = pd.DataFrame(data)

# Validar el DataFrame con el esquema
validated_df = ClientSchema.validate(df)

print(validated_df)


SchemaError: Column 'dni' failed element-wise validator number 0: <Check check_dni_positive> failure cases: -123456789